# Loading Responses

In [1]:
options(warn=1)
if(!require('lme4')) {
    install.packages('lme4', repos='https://cloud.r-project.org')
    library('lme4')
}
if(!require('readr')) {
    install.packages('readr', repos='https://cloud.r-project.org')
    library('readr')
}
if(!require('marginaleffects')) {
    install.packages('marginaleffects', repos='https://cloud.r-project.org')
    library('marginaleffects')
}
if(!require('emmeans')) {
    install.packages('emmeans', repos='https://cloud.r-project.org')
    library('emmeans')
}
if(!require('car')) {
    install.packages('car', repos='https://cloud.r-project.org')
    library('car')
}
df <- read_csv("result_1743457603_20250506_20250506F.csv", na = "empty", col_select = c("Accuracy", "Group", "GroupKind", "Angle", "Proximity", "n_candidates", "Actor", "Candidates", "Stimulus_ID", "Prompt_ID", "Participant_ID", "list_id", "Run_ID", "Part"), col_types = cols(
    Accuracy = col_logical(),
    Group = col_factor(),
    GroupKind = col_factor(),
    Angle = col_factor(),
    Proximity = col_integer(),
    n_candidates = col_integer(),
    Actor = col_factor(c('X', 'Y')),
    Candidates = col_factor(),
    Stimulus_ID = col_factor(),
    Prompt_ID = col_factor(),
    Participant_ID = col_factor(),
    list_id = col_factor(),
    Run_ID = col_character(),
    Part = col_character(),
),show_col_types = TRUE)
df <- df[df$Group == "gemini-1.5-pro" & df$list_id != -1 & df$Part != "p0", ]
df$offset <- log(1/df$n_candidates / (1 - 1/df$n_candidates))
#df$Proximity <- df$Proximity - mean(df$Proximity)
#df$n_candidates <- df$n_candidates - mean(df$n_candidates)

Loading required package: lme4

Loading required package: Matrix

Loading required package: readr

Loading required package: marginaleffects

Loading required package: emmeans

Welcome to emmeans.
Caution: You lose important information if you filter this package's results.
See '? untidy'

Loading required package: car

Loading required package: carData

Rows: 156825 Columns: 14
-- Column specification --------------------------------------------------------
Delimiter: ","
chr (2): Run_ID, Part
int (2): Proximity, n_candidates
lgl (1): Accuracy
fct (9): Stimulus_ID, Prompt_ID, Participant_ID, Group, GroupKind, Angle, Ac...

i Use `spec()` to retrieve the full column specification for this data.
i Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [2]:
sessionInfo()

R version 4.3.3 (2024-02-29)
Platform: aarch64-apple-darwin20.0.0 (64-bit)
Running under: macOS 15.5

Matrix products: default
BLAS/LAPACK: /Users/zory/miniforge3/envs/py310/lib/libopenblas.0.dylib;  LAPACK version 3.12.0

locale:
[1] C

time zone: America/Los_Angeles
tzcode source: system (macOS)

attached base packages:
[1] stats     graphics  grDevices utils     datasets  methods   base     

other attached packages:
[1] car_3.1-3              carData_3.0-5          emmeans_1.11.0        
[4] marginaleffects_0.25.1 readr_2.1.5            lme4_1.1-35.5         
[7] Matrix_1.6-5          

loaded via a namespace (and not attached):
 [1] bit_4.6.0          jsonlite_2.0.0     compiler_4.3.3     crayon_1.5.3      
 [5] tidyselect_1.2.1   Rcpp_1.0.14        IRdisplay_1.1      parallel_4.3.3    
 [9] splines_4.3.3      boot_1.3-31        uuid_1.2-1         fastmap_1.2.0     
[13] IRkernel_1.3.2     lattice_0.22-6     coda_0.19-4.1      R6_2.6.1          
[17] generics_0.1.4     Formula_1.2

# Fitting models

In [3]:
model0 <- glmer(Accuracy ~ Angle + scale(Proximity, scale=FALSE) + scale(n_candidates, scale=FALSE) + Actor + (1|Actor:Candidates) + (1|Actor:Candidates:Stimulus_ID) + (1|Prompt_ID), 
               data = df,
               family = binomial(link = "logit"), 
               offset = offset)
model0

Warning message in checkConv(attr(opt, "derivs"), opt$par, ctrl = control$checkConv, :
"Model failed to converge with max|grad| = 0.00227124 (tol = 0.002, component 1)"


Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: binomial  ( logit )
Formula: 
Accuracy ~ Angle + scale(Proximity, scale = FALSE) + scale(n_candidates,  
    scale = FALSE) + Actor + (1 | Actor:Candidates) + (1 | Actor:Candidates:Stimulus_ID) +  
    (1 | Prompt_ID)
   Data: df
 Offset: offset
      AIC       BIC    logLik  deviance  df.resid 
 7013.277  7077.222 -3497.639  6995.277      8991 
Random effects:
 Groups                       Name        Std.Dev. 
 Actor:Candidates:Stimulus_ID (Intercept) 4.0433874
 Actor:Candidates             (Intercept) 0.0002256
 Prompt_ID                    (Intercept) 0.1569625
Number of obs: 9000, groups:  
Actor:Candidates:Stimulus_ID, 900; Actor:Candidates, 18; Prompt_ID, 12
Fixed Effects:
                       (Intercept)                           Angleleft  
                            0.5441                             -0.4441  
                        Angleright     scale(Proximity, scale 

In [4]:
model1 <- glmer(Accuracy ~ Angle + scale(Proximity, scale=FALSE) + scale(n_candidates, scale=FALSE) + Actor + (1|Actor:Stimulus_ID) + (1|Prompt_ID), 
               data = df,
               family = binomial(link = "logit"), 
               offset = offset)
model1

Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: binomial  ( logit )
Formula: 
Accuracy ~ Angle + scale(Proximity, scale = FALSE) + scale(n_candidates,  
    scale = FALSE) + Actor + (1 | Actor:Stimulus_ID) + (1 | Prompt_ID)
   Data: df
 Offset: offset
      AIC       BIC    logLik  deviance  df.resid 
 7011.277  7068.117 -3497.639  6995.277      8992 
Random effects:
 Groups            Name        Std.Dev.
 Actor:Stimulus_ID (Intercept) 4.043   
 Prompt_ID         (Intercept) 0.157   
Number of obs: 9000, groups:  Actor:Stimulus_ID, 900; Prompt_ID, 12
Fixed Effects:
                       (Intercept)                           Angleleft  
                            0.5441                             -0.4442  
                        Angleright     scale(Proximity, scale = FALSE)  
                           -0.2524                             -0.7219  
scale(n_candidates, scale = FALSE)                              ActorY  
       

In [5]:
model2 <- glmer(Accuracy ~ Angle + scale(Proximity, scale=FALSE) + scale(n_candidates, scale=FALSE) + Actor + (1|Actor:Candidates) + (1|Actor:Candidates:Stimulus_ID), 
               data = df,
               family = binomial(link = "logit"), 
               offset = offset)
model2

boundary (singular) fit: see help('isSingular')



Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: binomial  ( logit )
Formula: 
Accuracy ~ Angle + scale(Proximity, scale = FALSE) + scale(n_candidates,  
    scale = FALSE) + Actor + (1 | Actor:Candidates) + (1 | Actor:Candidates:Stimulus_ID)
   Data: df
 Offset: offset
      AIC       BIC    logLik  deviance  df.resid 
 7017.499  7074.339 -3500.750  7001.499      8992 
Random effects:
 Groups                       Name        Std.Dev. 
 Actor:Candidates:Stimulus_ID (Intercept) 4.027e+00
 Actor:Candidates             (Intercept) 1.217e-05
Number of obs: 9000, groups:  
Actor:Candidates:Stimulus_ID, 900; Actor:Candidates, 18
Fixed Effects:
                       (Intercept)                           Angleleft  
                            0.5473                             -0.4451  
                        Angleright     scale(Proximity, scale = FALSE)  
                           -0.2507                             -0.7179  
scale(n

In [6]:
model3 <- glmer(Accuracy ~ Angle + scale(Proximity, scale=FALSE) + scale(n_candidates, scale=FALSE) + Actor + (1|Actor:Stimulus_ID), 
               data = df,
               family = binomial(link = "logit"), 
               offset = offset)
model3

Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: binomial  ( logit )
Formula: 
Accuracy ~ Angle + scale(Proximity, scale = FALSE) + scale(n_candidates,  
    scale = FALSE) + Actor + (1 | Actor:Stimulus_ID)
   Data: df
 Offset: offset
      AIC       BIC    logLik  deviance  df.resid 
 7015.499  7065.234 -3500.750  7001.499      8993 
Random effects:
 Groups            Name        Std.Dev.
 Actor:Stimulus_ID (Intercept) 4.027   
Number of obs: 9000, groups:  Actor:Stimulus_ID, 900
Fixed Effects:
                       (Intercept)                           Angleleft  
                            0.5473                             -0.4451  
                        Angleright     scale(Proximity, scale = FALSE)  
                           -0.2507                             -0.7179  
scale(n_candidates, scale = FALSE)                              ActorY  
                           -0.8970                             -0.1651  

In [7]:
model3_5 <- glmer(Accuracy ~ Angle + scale(Proximity, scale=FALSE) + scale(n_candidates, scale=FALSE) + Actor + (1|Actor:Candidates), 
               data = df,
               family = binomial(link = "logit"), 
               offset = offset)
model3_5

Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: binomial  ( logit )
Formula: 
Accuracy ~ Angle + scale(Proximity, scale = FALSE) + scale(n_candidates,  
    scale = FALSE) + Actor + (1 | Actor:Candidates)
   Data: df
 Offset: offset
      AIC       BIC    logLik  deviance  df.resid 
12024.321 12074.056 -6005.161 12010.321      8993 
Random effects:
 Groups           Name        Std.Dev.
 Actor:Candidates (Intercept) 0.1981  
Number of obs: 9000, groups:  Actor:Candidates, 18
Fixed Effects:
                       (Intercept)                           Angleleft  
                           0.47627                            -0.26027  
                        Angleright     scale(Proximity, scale = FALSE)  
                          -0.18684                            -0.20687  
scale(n_candidates, scale = FALSE)                              ActorY  
                           0.10384                            -0.05227  

In [8]:
model4 <- glm(Accuracy ~ Angle * scale(Proximity, scale=FALSE) * scale(n_candidates, scale=FALSE) * Actor, 
               data = df,
               family = binomial(link = "logit"), 
               offset = offset)
model4


Call:  glm(formula = Accuracy ~ Angle * scale(Proximity, scale = FALSE) * 
    scale(n_candidates, scale = FALSE) * Actor, family = binomial(link = "logit"), 
    data = df, offset = offset)

Coefficients:
                                                                         (Intercept)  
                                                                           0.5436877  
                                                                           Angleleft  
                                                                          -0.3251485  
                                                                          Angleright  
                                                                          -0.2606240  
                                                     scale(Proximity, scale = FALSE)  
                                                                          -0.3674851  
                                                  scale(n_candidates, scale = FALSE)  
          

In [9]:
model5 <- glmer(Accuracy ~ Angle + scale(Proximity, scale=FALSE) * Actor + scale(n_candidates, scale=FALSE) + (1|Stimulus_ID), 
               data = df,
               family = binomial(link = "logit"), 
               offset = offset)
model5

Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: binomial  ( logit )
Formula: Accuracy ~ Angle + scale(Proximity, scale = FALSE) * Actor +  
    scale(n_candidates, scale = FALSE) + (1 | Stimulus_ID)
   Data: df
 Offset: offset
      AIC       BIC    logLik  deviance  df.resid 
 7014.502  7071.342 -3499.251  6998.502      8992 
Random effects:
 Groups      Name        Std.Dev.
 Stimulus_ID (Intercept) 4.024   
Number of obs: 9000, groups:  Stimulus_ID, 900
Fixed Effects:
                           (Intercept)                               Angleleft  
                                0.5490                                 -0.4454  
                            Angleright         scale(Proximity, scale = FALSE)  
                               -0.2559                                 -1.0186  
                                ActorY      scale(n_candidates, scale = FALSE)  
                               -0.1653                           

# Model selection

In [10]:
anova(model1, model2)

,npar,AIC,BIC,logLik,deviance,Chisq,Df,Pr(>Chisq)
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<lgl>
model1,8,7011.277,7068.117,-3497.639,6995.277,NA,NA,NA
model2,8,7017.499,7074.339,-3500.750,7001.499,0,0,NA


In [11]:
anova(model1, model3)

,npar,AIC,BIC,logLik,deviance,Chisq,Df,Pr(>Chisq)
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
model3,7,7015.499,7065.234,-3500.750,7001.499,NA,NA,NA
model1,8,7011.277,7068.117,-3497.639,6995.277,6.222326,1,0.01261493


In [12]:
anova(model3, model3_5)

,npar,AIC,BIC,logLik,deviance,Chisq,Df,Pr(>Chisq)
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<lgl>
model3,7,7015.499,7065.234,-3500.750,7001.499,NA,NA,NA
model3_5,7,12024.321,12074.056,-6005.161,12010.321,0,0,NA


In [13]:
anova(model3, model4)

,npar,AIC,BIC,logLik,deviance,Chisq,Df,Pr(>Chisq)
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
model3,7,7015.499,7065.234,-3500.750,7001.499,NA,NA,NA
model4,24,12007.389,12177.908,-5979.694,11959.389,0,17,1


In [14]:
anova(model1, model5)

,npar,AIC,BIC,logLik,deviance,Chisq,Df,Pr(>Chisq)
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<lgl>
model1,8,7011.277,7068.117,-3497.639,6995.277,NA,NA,NA
model5,8,7014.502,7071.342,-3499.251,6998.502,0,0,NA


In [15]:
model <- model1

# Interpretation

In [16]:
summary(model)

Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: binomial  ( logit )
Formula: 
Accuracy ~ Angle + scale(Proximity, scale = FALSE) + scale(n_candidates,  
    scale = FALSE) + Actor + (1 | Actor:Stimulus_ID) + (1 | Prompt_ID)
   Data: df
 Offset: offset

     AIC      BIC   logLik deviance df.resid 
  7011.3   7068.1  -3497.6   6995.3     8992 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-3.1910 -0.1774 -0.0992  0.1699  3.1835 

Random effects:
 Groups            Name        Variance Std.Dev.
 Actor:Stimulus_ID (Intercept) 16.34975 4.043   
 Prompt_ID         (Intercept)  0.02464 0.157   
Number of obs: 9000, groups:  Actor:Stimulus_ID, 900; Prompt_ID, 12

Fixed effects:
                                   Estimate Std. Error z value Pr(>|z|)    
(Intercept)                          0.5441     0.3072   1.771 0.076502 .  
Angleleft                           -0.4442     0.3813  -1.165 0.244011    
Angleright            

In [17]:
vc_list <- lapply(VarCorr(model), function(x) attr(x, "stddev")^2)
vc <- sum(unlist(vc_list))
effect_size_denom_t <- sqrt(vc + pi^2/3)

res <- summary(model)$coefficients
res <- as.data.frame(res)
res <- cbind(term = rownames(res), res)
rownames(res) <- NULL
res$`Pr(>|z|)` <- format(res$`Pr(>|z|)`, scientific = FALSE)
res$CI_low <- res$Estimate - 1.96 * res$`Std. Error`
res$CI_high <- res$Estimate + 1.96 * res$`Std. Error`
res$effect_size_denom <- effect_size_denom_t
res$effect_size <- res$Estimate / effect_size_denom_t
res

term,Estimate,Std. Error,z value,Pr(>|z|),CI_low,CI_high,effect_size_denom,effect_size
<chr>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>
(Intercept),0.5441388,0.3071881,1.7713536,0.07650192169,-0.05794994,1.1462276,4.43444,0.12270746
Angleleft,-0.4442179,0.3812962,-1.1650204,0.24401074234,-1.19155845,0.3031227,4.43444,-0.10017452
Angleright,-0.2523940,0.3824308,-0.6599730,0.50927112581,-1.00195826,0.4971703,4.43444,-0.05691677
"scale(Proximity, scale = FALSE)",-0.7219144,0.1929891,-3.7406999,0.00018350854,-1.10017315,-0.3436557,4.43444,-0.16279722
"scale(n_candidates, scale = FALSE)",-0.9059535,0.2076576,-4.3627270,0.00001284512,-1.31296237,-0.4989445,4.43444,-0.20429943
ActorY,-0.1659478,0.3145730,-0.5275335,0.59782312481,-0.78251082,0.4506152,4.43444,-0.03742250


In [18]:
avg_predictions(model, by="Angle", hypothesis = ~ pairwise)

hypothesis,estimate,std.error,statistic,p.value,s.value,conf.low,conf.high
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
(left) - (front),-0.05120185,0.03515357,-1.4565191,0.1452492,2.783398,-0.12010158,0.01769788
(right) - (front),-0.02459702,0.03526152,-0.6975599,0.4854525,1.042598,-0.09370833,0.04451429
(right) - (left),0.02660482,0.03764574,0.7067154,0.4797434,1.059665,-0.04717947,0.10038912


In [19]:
avg_predictions(model, by="Proximity")

Proximity,estimate,std.error,statistic,p.value,s.value,conf.low,conf.high
<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,0.5788503,0.02340334,24.73366,4.647697e-135,446.2438,0.5329806,0.6247200
2,0.5170300,0.01542890,33.51048,3.390954e-246,815.4326,0.4867899,0.5472701
3,0.4750114,0.02408671,19.72089,1.426911e-86,285.1729,0.4278023,0.5222205


In [20]:
avg_slopes(model, variables ="Proximity", type = "link")

term,contrast,estimate,std.error,statistic,p.value,s.value,conf.low,conf.high,predicted_lo,predicted_hi,predicted
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Proximity,dY/dX,-0.7219144,0.1929876,-3.74073,0.0001834868,12.41204,-1.100163,-0.3436657,-3.662671,-3.662815,-3.662743


In [21]:
avg_predictions(model, by="n_candidates")

n_candidates,estimate,std.error,statistic,p.value,s.value,conf.low,conf.high
<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
2,0.5581318,0.01856451,30.06445,1.413628e-198,657.2424,0.5217461,0.5945176
3,0.5150404,0.01655218,31.11616,1.456022e-212,703.7067,0.4825987,0.5474821
4,0.4169142,0.03539239,11.77977,4.962883e-32,103.9905,0.3475464,0.4862820


In [22]:
avg_slopes(model, variables ="n_candidates", type = "link")

term,contrast,estimate,std.error,statistic,p.value,s.value,conf.low,conf.high,predicted_lo,predicted_hi,predicted
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
n_candidates,dY/dX,-0.9059535,0.207659,-4.362698,1.284684e-05,16.24823,-1.312958,-0.4989493,-3.662652,-3.662834,-3.662743


The below code is for generating the table in the paper.

In [27]:
stim_id_var_val <- if (!is.null(vc_list$`Actor:Stimulus_ID`) && length(vc_list$`Actor:Stimulus_ID`) > 0) vc_list$`Actor:Stimulus_ID`[[1]] else NA_real_
prompt_id_var_val <- if (!is.null(vc_list$Prompt_ID) && length(vc_list$Prompt_ID) > 0) vc_list$Prompt_ID[[1]] else NA_real_
participant_id_var_val <- if (!is.null(vc_list$Participant_ID) && length(vc_list$Participant_ID) > 0) vc_list$Participant_ID[[1]] else NA_real_

intercept_est <- res[res$term == "(Intercept)", "Estimate"]
intercept_se <- res[res$term == "(Intercept)", "Std. Error"]

angleleft_est <- res[res$term == "Angleleft", "Estimate"]
angleleft_se <- res[res$term == "Angleleft", "Std. Error"]
angleleft_p_val <- res[res$term == "Angleleft", "Pr(>|z|)"]
angleleft_eff_size <- res[res$term == "Angleleft", "effect_size"]

angleright_est <- res[res$term == "Angleright", "Estimate"]
angleright_se <- res[res$term == "Angleright", "Std. Error"]
angleright_p_val <- res[res$term == "Angleright", "Pr(>|z|)"]
angleright_eff_size <- res[res$term == "Angleright", "effect_size"]

proximity_term_name <- "scale(Proximity, scale = FALSE)"
if (!proximity_term_name %in% res$term) {
    proximity_term_name <- grep("Proximity", res$term, value = TRUE)[1]
}
proximity_est <- res[res$term == proximity_term_name, "Estimate"]
proximity_se <- res[res$term == proximity_term_name, "Std. Error"]
proximity_p_val <- res[res$term == proximity_term_name, "Pr(>|z|)"]
proximity_eff_size <- res[res$term == proximity_term_name, "effect_size"]

n_candidates_term_name <- "scale(n_candidates, scale = FALSE)"
if (!n_candidates_term_name %in% res$term) {
    n_candidates_term_name <- grep("n_candidates", res$term, value = TRUE)[1]
}
n_obj_est <- res[res$term == n_candidates_term_name, "Estimate"]
n_obj_se <- res[res$term == n_candidates_term_name, "Std. Error"]
n_obj_p_val <- res[res$term == n_candidates_term_name, "Pr(>|z|)"]
n_obj_eff_size <- res[res$term == n_candidates_term_name, "effect_size"]

actory_term_name <- "ActorY" # Assuming 'X' is the reference level for Actor factor
actory_est <- res[res$term == actory_term_name, "Estimate"]
actory_se <- res[res$term == actory_term_name, "Std. Error"]
actory_p_val <- res[res$term == actory_term_name, "Pr(>|z|)"]
actory_eff_size <- res[res$term == actory_term_name, "effect_size"]

# Prepare data for data frame
table_data <- list(
    c("StimulusID", "Variance", sprintf("%.7f", stim_id_var_val)),
    c("ParticipantID", "Variance", sprintf("%.7f", participant_id_var_val)),
    c("PromptID", "Variance", sprintf("%.7f", prompt_id_var_val)),
    c("Effect Size Denom", "", sprintf("%.7f", effect_size_denom_t)),

    c("Intercept", "b", sprintf("%.7f", intercept_est)),
    c("", "SE", sprintf("%.7f", intercept_se)),

    c("Angleleft", "b", sprintf("%.7f", angleleft_est)),
    c("", "SE", sprintf("%.7f", angleleft_se)),
    c("", "p", sprintf("%.7f", as.numeric(angleleft_p_val))),
    c("", "Effect size", sprintf("%.7f", angleleft_eff_size)),

    c("Angleright", "b", sprintf("%.7f", angleright_est)),
    c("", "SE", sprintf("%.7f", angleright_se)),
    c("", "p", sprintf("%.7f", as.numeric(angleright_p_val))),
    c("", "Effect size", sprintf("%.7f", angleright_eff_size)),

    c("Proximity", "b", sprintf("%.7f", proximity_est)),
    c("", "SE", sprintf("%.7f", proximity_se)),
    c("", "p", sprintf("%.7f", as.numeric(proximity_p_val))),
    c("", "Effect size", sprintf("%.7f", proximity_eff_size)),

    c("#Object", "b", sprintf("%.7f", n_obj_est)),
    c("", "SE", sprintf("%.7f", n_obj_se)),
    c("", "p", sprintf("%.7f", as.numeric(n_obj_p_val))),
    c("", "Effect size", sprintf("%.7f", n_obj_eff_size)),

    c("ActorY", "b", sprintf("%.7f", actory_est)),
    c("", "SE", sprintf("%.7f", actory_se)),
    c("", "p", sprintf("%.7f", as.numeric(actory_p_val))),
    c("", "Effect size", sprintf("%.7f", actory_eff_size))
)

# Convert to data frame
df_output <- do.call(rbind, lapply(table_data, function(x) data.frame(V1=x[1], V2=x[2], V3=x[3], stringsAsFactors=FALSE)))

# Print only the third column for easy copy-pasting to Excel
write.table(df_output[,3, drop=FALSE], file="", sep = "\t", row.names = FALSE, col.names = FALSE, quote = FALSE)

16.3497467
NA
0.0246394
4.4344396
0.5441388
0.3071881
-0.4442179
0.3812962
0.2440107
-0.1001745
-0.2523940
0.3824308
0.5092711
-0.0569168
-0.7219144
0.1929891
0.0001835
-0.1627972
-0.9059535
0.2076576
0.0000128
-0.2042994
-0.1659478
0.3145730
0.5978231
-0.0374225
